In [1]:
import numpy as np
import pandas as pd
import patsy

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.cross_validation import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.grid_search import GridSearchCV

//anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
//anaconda/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


# ![](https://ga-dash.s3.amazonaws.com/production/assets/logo-9f88ae6c9c3871690e33280fcf557f33.png) Advanced Model Tuning

Week 5 | Day 1

### LEARNING OBJECTIVES
*After this lesson, you will be able to:*
- Explain what gridsearch is and why it's useful in machine learning
- Implement auto-tuning in sklearn

## Recap

At this point we have learned about a couple of differenct classification models.


Check: Which models do we know know?

## Recap

And we've seen that each of these models have inputs that can be set upon initialization

Check: What are some of these inputs? Take a quick look...
    
[K-NN](http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html)<br>
[Logistic Regression](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)

## Given all the possible inputs, how can we fit the best model?

## Autotune!


It isn't realistic for us to attempt to construct and all the possible combinations of models by hand.

Fortunately, sklearn provides a number of built-in functions, that allow us to search the space of all possible models to find the best one. (Recall LassoCV and RidgeCV)

We're going to walk through an example of how to do that now step by step...

## We are going to build a model to predict crime in SF


## The features will included day of week, time, and district
## The target is the type of crime

## Load our data set 

In [2]:
sf_crime = pd.read_csv('./assets/datasets/sf_crime_train.csv')
sf_crime = sf_crime.dropna()

In [3]:
sf_crime.head()

,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y
0,5/13/15 23:53,WARRANTS,WARRANT ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
1,5/13/15 23:53,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
2,5/13/15 23:33,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",VANNESS AV / GREENWICH ST,-122.424363,37.800414
3,5/13/15 23:30,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,NORTHERN,NONE,1500 Block of LOMBARD ST,-122.426995,37.800873
4,5/13/15 23:30,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,PARK,NONE,100 Block of BRODERICK ST,-122.438738,37.771541


## Data type conversions and transformations

In [4]:
sf_crime['Dates'] = pd.to_datetime(sf_crime['Dates'])
sf_crime_dates = pd.DatetimeIndex(sf_crime['Dates'].values, dtype='datetime64[ns]', freq=None)

sf_crime['hour'] = sf_crime_dates.hour
sf_crime['month'] = sf_crime_dates.month
sf_crime['year'] = sf_crime_dates.year

## Let's see what all the listed crimes are

In [5]:
sf_crime['Category'].unique()

array(['WARRANTS', 'OTHER OFFENSES', 'LARCENY/THEFT', 'VEHICLE THEFT',
       'VANDALISM', 'NON-CRIMINAL', 'ROBBERY', 'ASSAULT', 'WEAPON LAWS',
       'BURGLARY', 'SUSPICIOUS OCC', 'DRUNKENNESS',
       'FORGERY/COUNTERFEITING', 'DRUG/NARCOTIC', 'STOLEN PROPERTY',
       'SECONDARY CODES', 'TRESPASS', 'MISSING PERSON', 'FRAUD',
       'KIDNAPPING', 'RUNAWAY', 'DRIVING UNDER THE INFLUENCE',
       'SEX OFFENSES FORCIBLE', 'PROSTITUTION', 'DISORDERLY CONDUCT',
       'ARSON', 'FAMILY OFFENSES', 'LIQUOR LAWS', 'BRIBERY',
       'EMBEZZLEMENT', 'SUICIDE', 'LOITERING', 'SEX OFFENSES NON FORCIBLE',
       'EXTORTION', 'GAMBLING', 'BAD CHECKS'], dtype=object)

## We'll select a subsection of the listed crimes

In [6]:
subset = ['VEHICLE THEFT','BURGLARY','DRUG/NARCOTIC']
sf_crime_sub = sf_crime[sf_crime['Category'].isin(subset)]

In [7]:
sf_crime_sub.head()

,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y,hour,month,year
6,2015-05-13 23:30:00,VEHICLE THEFT,STOLEN AUTOMOBILE,Wednesday,INGLESIDE,NONE,AVALON AV / PERU AV,-122.423327,37.725138,23,5,2015
7,2015-05-13 23:30:00,VEHICLE THEFT,STOLEN AUTOMOBILE,Wednesday,BAYVIEW,NONE,KIRKWOOD AV / DONAHUE ST,-122.371274,37.727564,23,5,2015
46,2015-05-13 20:00:00,VEHICLE THEFT,STOLEN MOTORCYCLE,Wednesday,INGLESIDE,NONE,0 Block of CRESCENT AV,-122.423702,37.735233,20,5,2015
49,2015-05-13 19:52:00,BURGLARY,"BURGLARY, VEHICLE (ARREST MADE)",Wednesday,PARK,"ARREST, BOOKED",1500 Block of HAIGHT ST,-122.447761,37.769846,19,5,2015
59,2015-05-13 19:28:00,VEHICLE THEFT,STOLEN AND RECOVERED VEHICLE,Wednesday,CENTRAL,NONE,0 Block of SANSOME ST,-122.400720,37.790712,19,5,2015


## Check the total number of districts

In [8]:
sf_crime_sub['PdDistrict'].unique()

array(['INGLESIDE', 'BAYVIEW', 'PARK', 'CENTRAL', 'MISSION', 'SOUTHERN',
       'NORTHERN', 'RICHMOND', 'TARAVAL', 'TENDERLOIN'], dtype=object)

In [9]:
sf_crime_sub['PdDistrict'].nunique()

10

## Set up our design matrix and target vector with Patsy

### Patsy allows us to use R-style formulas to do this 
[Patsy Docs](http://patsy.readthedocs.io/en/latest/)

In [10]:
X = patsy.dmatrix('~ C(hour) + C(DayOfWeek) + C(PdDistrict)', sf_crime_sub)
y = sf_crime_sub['Category'].values

In [11]:
y

array(['VEHICLE THEFT', 'VEHICLE THEFT', 'VEHICLE THEFT', ...,
       'VEHICLE THEFT', 'BURGLARY', 'VEHICLE THEFT'], dtype=object)

In [12]:
X.design_info.column_names

['Intercept',
 'C(hour)[T.1]',
 'C(hour)[T.2]',
 'C(hour)[T.3]',
 'C(hour)[T.4]',
 'C(hour)[T.5]',
 'C(hour)[T.6]',
 'C(hour)[T.7]',
 'C(hour)[T.8]',
 'C(hour)[T.9]',
 'C(hour)[T.10]',
 'C(hour)[T.11]',
 'C(hour)[T.12]',
 'C(hour)[T.13]',
 'C(hour)[T.14]',
 'C(hour)[T.15]',
 'C(hour)[T.16]',
 'C(hour)[T.17]',
 'C(hour)[T.18]',
 'C(hour)[T.19]',
 'C(hour)[T.20]',
 'C(hour)[T.21]',
 'C(hour)[T.22]',
 'C(hour)[T.23]',
 'C(DayOfWeek)[T.Monday]',
 'C(DayOfWeek)[T.Saturday]',
 'C(DayOfWeek)[T.Sunday]',
 'C(DayOfWeek)[T.Thursday]',
 'C(DayOfWeek)[T.Tuesday]',
 'C(DayOfWeek)[T.Wednesday]',
 'C(PdDistrict)[T.CENTRAL]',
 'C(PdDistrict)[T.INGLESIDE]',
 'C(PdDistrict)[T.MISSION]',
 'C(PdDistrict)[T.NORTHERN]',
 'C(PdDistrict)[T.PARK]',
 'C(PdDistrict)[T.RICHMOND]',
 'C(PdDistrict)[T.SOUTHERN]',
 'C(PdDistrict)[T.TARAVAL]',
 'C(PdDistrict)[T.TENDERLOIN]']

## Let's look at our design matrix as a DataFrame

In [13]:
pdf = pd.DataFrame(X, columns=X.design_info.column_names)
pdf['Target'] = y
pdf

,Intercept,C(hour)[T.1],C(hour)[T.2],C(hour)[T.3],C(hour)[T.4],C(hour)[T.5],C(hour)[T.6],C(hour)[T.7],C(hour)[T.8],C(hour)[T.9],...,C(PdDistrict)[T.CENTRAL],C(PdDistrict)[T.INGLESIDE],C(PdDistrict)[T.MISSION],C(PdDistrict)[T.NORTHERN],C(PdDistrict)[T.PARK],C(PdDistrict)[T.RICHMOND],C(PdDistrict)[T.SOUTHERN],C(PdDistrict)[T.TARAVAL],C(PdDistrict)[T.TENDERLOIN],Target
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,VEHICLE THEFT
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,VEHICLE THEFT
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,VEHICLE THEFT
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,BURGLARY
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,VEHICLE THEFT
5,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,VEHICLE THEFT
6,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,VEHICLE THEFT
7,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BURGLARY
8,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,BURGLARY
9,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,BURGLARY


## Let's see how many districts are listed in our design matrix 

In [14]:
sf_crime_sub['PdDistrict'].nunique()

10

In [15]:
[x for x in pdf.columns if 'PdDistrict' in x]

['C(PdDistrict)[T.CENTRAL]',
 'C(PdDistrict)[T.INGLESIDE]',
 'C(PdDistrict)[T.MISSION]',
 'C(PdDistrict)[T.NORTHERN]',
 'C(PdDistrict)[T.PARK]',
 'C(PdDistrict)[T.RICHMOND]',
 'C(PdDistrict)[T.SOUTHERN]',
 'C(PdDistrict)[T.TARAVAL]',
 'C(PdDistrict)[T.TENDERLOIN]']

In [16]:
pd.Series([x for x in pdf.columns if 'PdDistrict' in x]).nunique()

9

## And how many hours?

In [17]:
sf_crime_sub['hour'].nunique()

24

In [18]:
pd.Series([x for x in pdf.columns if 'hour' in x]).nunique()

23

## Check: Why is there one less on both?

## Set up our training and testing sets

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=77)

## Now let's fit a standard logistic regression model

In [20]:
lr = LogisticRegression(solver='liblinear')

In [21]:
lr_model = lr.fit(X_train, y_train)

## Make our predictions

In [22]:
lr_ypred = lr_model.predict(X_test)

## Check our misclassifications with a confusion matrix

[Confusion Matrix](http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html#sphx-glr-auto-examples-model-selection-plot-confusion-matrix-py)

In [23]:
# actual = index; predicted = columns
lr_cm = confusion_matrix(y_test, lr_ypred, labels=lr.classes_)
lr_cm = pd.DataFrame(lr_cm, columns=lr.classes_, index=lr.classes_)
lr_cm

,BURGLARY,DRUG/NARCOTIC,VEHICLE THEFT
BURGLARY,96,19,134
DRUG/NARCOTIC,45,49,76
VEHICLE THEFT,59,11,236


## Check our precision, recall, and f1

[Precision & Recall](https://www.quora.com/What-is-the-best-way-to-understand-the-terms-precision-and-recall)<br>
[F1](https://en.wikipedia.org/wiki/F1_score)

In [24]:
print classification_report(y_test, lr_ypred, labels=lr.classes_)

               precision    recall  f1-score   support

     BURGLARY       0.48      0.39      0.43       249
DRUG/NARCOTIC       0.62      0.29      0.39       170
VEHICLE THEFT       0.53      0.77      0.63       306

  avg / total       0.53      0.53      0.50       725



## Check the CV Score

In [25]:
cvs1 = cross_val_score(lr, X, y, cv=3, scoring='f1_weighted')
cvs1

array([ 0.44947491,  0.50286206,  0.54306079])

In [26]:
cvs1.mean()

0.49846591887492991

## Let's now use a penalized regression - we'll use LASSO (L1)

In [27]:
lr_l1 = LogisticRegression(C=1, penalty='l1', solver='liblinear')
lr_l1_model = lr_l1.fit(X_train, y_train)

In [28]:
lr_l1_model = lr_l1.fit(X_train, y_train)

In [29]:
lr_l1_ypred = lr_l1_model.predict(X_test)

## Get the confusion matrix

In [30]:
lr_l1_cm = confusion_matrix(y_test, lr_l1_ypred, labels=lr_l1.classes_)
lr_l1_cm = pd.DataFrame(lr_l1_cm, columns=lr_l1.classes_, index=lr_l1.classes_)
lr_l1_cm

,BURGLARY,DRUG/NARCOTIC,VEHICLE THEFT
BURGLARY,91,18,140
DRUG/NARCOTIC,45,47,78
VEHICLE THEFT,63,10,233


## Get the classification report

In [31]:
print classification_report(y_test, lr_l1_ypred, labels=lr_l1.classes_)

               precision    recall  f1-score   support

     BURGLARY       0.46      0.37      0.41       249
DRUG/NARCOTIC       0.63      0.28      0.38       170
VEHICLE THEFT       0.52      0.76      0.62       306

  avg / total       0.52      0.51      0.49       725



## Get mean cross val score

In [32]:
cvs2 = cross_val_score(lr_l1, X, y, cv=3)

In [33]:
cvs2.mean()

0.51257287076764779

 ## Looks like a minimal + change with L1 penalty at 1, how about other values?

## We can build a function to test this

In [34]:
def test_penalties(c_val):
    lr_l1 = LogisticRegression(C=c_val, penalty='l1', solver='liblinear')
    cvs = cross_val_score(lr_l1, X, y, cv=3, scoring='f1_weighted')
    return cvs

In [35]:
# let's test it...
test_cs = pd.Series([.001, .01, .1, 1, 1.5, 2.5, 5, 10, 100]).to_frame('c_vals')
score_frame = pd.DataFrame([test_penalties(x) for x in test_cs['c_vals']]).mean(axis=1).to_frame('score')

final_scores = pd.concat([test_cs, score_frame], axis=1)
final_scores

//anaconda/lib/python2.7/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


,c_vals,score
0,0.001,0.166800
1,0.010,0.269455
2,0.100,0.415842
3,1.000,0.495965
4,1.500,0.500745
5,2.500,0.499080
6,5.000,0.505656
7,10.000,0.501893
8,100.000,0.500465


In [36]:
# and so the best c value...
final_scores['c_vals'][final_scores['score'].idxmax()]

5.0

## That wasn't too bad, but...

But that was only changing one parameter. What if we wanted to try out L2 as well as L1? Or if we had a different algorithm with numerous parameters? It can start to become a hassle to code these from scratch.

Fortunately, sk-learn has a function that will do this for us.

In [37]:
# fit model with three folds and lasso regularization
# use Cs=20 to test a grid of 20 distinct parameters
# remeber: Cs describes the inverse of regularization strength
logreg_cv = LogisticRegressionCV(Cs=20, solver='liblinear', cv=3, penalty='l1', scoring='f1')
cv_model = logreg_cv.fit(X_train, y_train)

//anaconda/lib/python2.7/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


## Find best C per class

In [38]:
print('best C for class:')
best_C = {logreg_cv.classes_[i]:x for i, (x, c) in enumerate(zip(logreg_cv.Cs_, logreg_cv.classes_))}
print(best_C)

best C for class:
{'BURGLARY': 0.0001, 'VEHICLE THEFT': 0.00069519279617756048, 'DRUG/NARCOTIC': 0.00026366508987303583}


## Get the classification report for our best model

In [39]:
print classification_report(y_test, logreg_cv.predict(X_test))

               precision    recall  f1-score   support

     BURGLARY       0.48      0.40      0.44       249
DRUG/NARCOTIC       0.69      0.25      0.36       170
VEHICLE THEFT       0.53      0.78      0.63       306

  avg / total       0.55      0.53      0.50       725



## Exercise

Using the data set (pdf):
- Fit two models to predict between "Burglary" and "Drug/Narcotic" crimes
- One model should use an L1 penalty and the other should use an L2 penalty
- Make sure to use train_test_split
- Print out a confusion matrix and a classification report for both models
- Finally, build a third model that uses LogisticRegressionCV
- Print our a confusion matrix, classification report and the best value of C

In [46]:
pdf['Target'].unique()

array(['VEHICLE THEFT', 'BURGLARY', 'DRUG/NARCOTIC'], dtype=object)

In [48]:
pdf2 = pdf.copy()
pdf2 = pdf2[pdf2['Target']!='VEHICLE THEFT']

In [50]:
X2 = pdf2[pdf.columns[:-1]]
y2 = pdf2['Target']

In [51]:
X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, y2, test_size=0.33, random_state=77)

In [52]:
lr3 = LogisticRegression()
m3 = lr3.fit(X_train2, y_train2)

In [54]:
cm3 = confusion_matrix(y_test2, m3.predict(X_test2), labels=m3.classes_)
cm3 = pd.DataFrame(cm3, columns=m3.classes_, index=m3.classes_)
cm3

,BURGLARY,DRUG/NARCOTIC
BURGLARY,212,37
DRUG/NARCOTIC,91,66


In [53]:
print classification_report(y_test2, m3.predict(X_test2))

               precision    recall  f1-score   support

     BURGLARY       0.70      0.85      0.77       249
DRUG/NARCOTIC       0.64      0.42      0.51       157

  avg / total       0.68      0.68      0.67       406



In [55]:
lr4 = LogisticRegression(penalty='l1')
m4 = lr4.fit(X_train2, y_train2)

In [56]:
cm4 = confusion_matrix(y_test2, m4.predict(X_test2), labels=m4.classes_)
cm4 = pd.DataFrame(cm4, columns=m4.classes_, index=m4.classes_)
cm4

,BURGLARY,DRUG/NARCOTIC
BURGLARY,221,28
DRUG/NARCOTIC,95,62


In [57]:
print classification_report(y_test2, m4.predict(X_test2))

               precision    recall  f1-score   support

     BURGLARY       0.70      0.89      0.78       249
DRUG/NARCOTIC       0.69      0.39      0.50       157

  avg / total       0.70      0.70      0.67       406



In [58]:
lr5 = LogisticRegressionCV(Cs=20, solver='liblinear', cv=3, penalty='l1', scoring='f1')
m5 = lr5.fit(X_train2, y_train2)

In [59]:
cm5 = confusion_matrix(y_test2, m5.predict(X_test2), labels=m5.classes_)
cm5 = pd.DataFrame(cm5, columns=m5.classes_, index=m5.classes_)
cm5

,BURGLARY,DRUG/NARCOTIC
BURGLARY,209,40
DRUG/NARCOTIC,88,69


In [60]:
print classification_report(y_test2, m5.predict(X_test2))

               precision    recall  f1-score   support

     BURGLARY       0.70      0.84      0.77       249
DRUG/NARCOTIC       0.63      0.44      0.52       157

  avg / total       0.68      0.68      0.67       406



In [65]:
print 'best C: ', m5.C_

best C:  [ 4.2813324]


In [61]:
print('best C for class:')
best_C2 = {m5.classes_[i]:x for i, (x, c) in enumerate(zip(m5.Cs_, m5.classes_))}
print(best_C2)

best C for class:
{'BURGLARY': 0.0001, 'DRUG/NARCOTIC': 0.00026366508987303583}


## So LogisticRegressionCV is useful for finding the best penalty, but we had to manually change our penalty from 'l1' to 'l2' to try both...What if there was a better way...?

## Introducing GridSearchCV

[GridSearchCV](http://scikit-learn.org/0.17/modules/generated/sklearn.grid_search.GridSearchCV.html)

## To start we'll select a model and penalties and some hyperparameters 

Then will pass those to GridSearchCV

In [66]:
logreg = LogisticRegression(solver='liblinear')
C_vals = [0.0001, 0.001, 0.01, 0.1, .15, .25, .275, .33, 0.5, .66, 0.75, 1.0, 2.5, 5.0, 10.0, 100.0, 1000.0]
penalties = ['l1','l2']

gs = GridSearchCV(logreg, {'penalty': penalties, 'C': C_vals}, verbose=False, cv=15)
gs.fit(X, y)

GridSearchCV(cv=15, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'penalty': ['l1', 'l2'], 'C': [0.0001, 0.001, 0.01, 0.1, 0.15, 0.25, 0.275, 0.33, 0.5, 0.66, 0.75, 1.0, 2.5, 5.0, 10.0, 100.0, 1000.0]},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=False)

## Now let's find the best parameters

In [67]:
gs.best_params_

{'C': 0.1, 'penalty': 'l2'}

## Use this parameter to fit, predict, and print a classification_report for our X and Y

In [69]:
logreg = LogisticRegression(C=gs.best_params_['C'], penalty=gs.best_params_['penalty'])
cv_model = logreg.fit(X_train2, y_train2)

In [71]:
cv_pred = cv_model.predict(X_test2)

## Now let's check our stats...

In [72]:
cm3 = confusion_matrix(y_test2, cv_pred, labels=logreg.classes_)
cm3 = pd.DataFrame(cm3, columns=logreg.classes_, index=logreg.classes_)

In [73]:
cm3

,BURGLARY,DRUG/NARCOTIC
BURGLARY,229,20
DRUG/NARCOTIC,107,50


In [75]:
print classification_report(y_test2, cv_pred, labels=logreg.classes_)

               precision    recall  f1-score   support

     BURGLARY       0.68      0.92      0.78       249
DRUG/NARCOTIC       0.71      0.32      0.44       157

  avg / total       0.69      0.69      0.65       406



## Independent Practice

Use GridSearchCV with knn on the iris data set:
- Use train_test_split with a train size of .66
- Set a parameter dictionary with the number of neighbors and at least one other parameter
- Get your best estimator and print out a classification report

In [90]:
from sklearn import datasets
from sklearn.neighbors import KNeighborsClassifier

In [77]:
iris = datasets.load_iris()

In [84]:
features = pd.DataFrame(iris.data, columns=iris.feature_names)

In [85]:
features.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [86]:
target = iris.target

In [89]:
X_train3, X_test3, y_train3, y_test3 = train_test_split(features, target,\
                                                        train_size=0.66, stratify=target, random_state=77)

In [117]:
knn = KNeighborsClassifier()
neighbors = np.arange(1,31,1)
weights_o = ['uniform', 'distance']
leaf = np.arange(10,50,10)

In [118]:
params = dict(n_neighbors=neighbors, weights=weights_o, leaf_size = leaf)

In [120]:
params

{'leaf_size': array([10, 20, 30, 40]),
 'n_neighbors': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]),
 'weights': ['uniform', 'distance']}

In [119]:
gs2 = GridSearchCV(knn, params, verbose=False, cv=15, scoring='accuracy')
gs2.fit(X_train3, y_train3)

GridSearchCV(cv=15, error_score='raise',
       estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform'),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'n_neighbors': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]), 'weights': ['uniform', 'distance'], 'leaf_size': array([10, 20, 30, 40])},
       pre_dispatch='2*n_jobs', refit=True, scoring='accuracy',
       verbose=False)

In [129]:
gs2.best_estimator_

KNeighborsClassifier(algorithm='auto', leaf_size=10, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=9, p=2,
           weights='uniform')

In [121]:
gs2.best_params_

{'leaf_size': 10, 'n_neighbors': 9, 'weights': 'uniform'}

In [130]:
gs2.best_score_

0.9696969696969697

In [132]:
gs2.predict(X_test3)

array([1, 0, 0, 0, 1, 2, 2, 1, 2, 0, 1, 1, 2, 2, 0, 1, 0, 1, 1, 1, 0, 1, 2,
       0, 0, 1, 0, 2, 0, 2, 0, 1, 1, 1, 1, 2, 2, 1, 2, 0, 2, 1, 2, 1, 0, 2,
       0, 2, 0, 2, 0])

In [122]:
knn2 = KNeighborsClassifier(leaf_size= 10, n_neighbors= 9, weights= 'uniform')
knn2.fit(X_train3, y_train3)
pred = knn2.predict(X_test3)

In [123]:
pred

array([1, 0, 0, 0, 1, 2, 2, 1, 2, 0, 1, 1, 2, 2, 0, 1, 0, 1, 1, 1, 0, 1, 2,
       0, 0, 1, 0, 2, 0, 2, 0, 1, 1, 1, 1, 2, 2, 1, 2, 0, 2, 1, 2, 1, 0, 2,
       0, 2, 0, 2, 0])

In [124]:
y_test3

array([1, 0, 0, 0, 1, 2, 2, 1, 2, 0, 1, 1, 2, 2, 0, 1, 0, 1, 1, 1, 0, 2, 2,
       0, 0, 1, 0, 2, 0, 1, 0, 1, 2, 1, 1, 2, 2, 1, 2, 0, 2, 1, 2, 1, 0, 2,
       0, 2, 0, 2, 0])

In [125]:
cm6 = confusion_matrix(y_test3, pred, labels=knn2.classes_)
cm6 = pd.DataFrame(cm6, columns=knn2.classes_, index=knn2.classes_)

In [126]:
cm6

,0,1,2
0,17,0,0
1,0,16,1
2,0,2,15


In [127]:
iris.target_names

array(['setosa', 'versicolor', 'virginica'], 
      dtype='|S10')

In [128]:
print classification_report(y_test3, pred, labels=knn2.classes_)

             precision    recall  f1-score   support

          0       1.00      1.00      1.00        17
          1       0.89      0.94      0.91        17
          2       0.94      0.88      0.91        17

avg / total       0.94      0.94      0.94        51

